<a href="https://colab.research.google.com/github/zackives/upenn-cis-2450/blob/main/10_Module_2_Notebook_VI_PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raning Graph Data

Let's look at link analysis!

## Autograder setup

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [ ]:
%set_env HW_ID=cis2450_fall24_HW9

env: HW_ID=cis2450_fall24_HW9


In [ ]:
!pip3 install penngrader-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.0 MB/s eta 0:00:00


In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 99999999

Make sure this correct or we will not be able to store your grade


## Load LinkedIn Graph

In [ ]:
!wget https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl

--2024-10-08 17:14:47--  https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.207, 142.251.163.207, 142.251.167.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179851696 (172M) [application/octet-stream]
Saving to: ‘linkedin_anon.jsonl’

linkedin_anon.jsonl 100%[===================>] 171.52M   109MB/s    in 1.6s    

2024-10-08 17:14:49 (109 MB/s) - ‘linkedin_anon.jsonl’ saved [179851696/179851696]



In [ ]:
import pandas as pd

people_df = pd.read_json('linkedin_anon.jsonl', lines=True, nrows=5000)

In [ ]:
# Replace the index with a numeric one
people_df['_id'] = people_df.index

In [ ]:
def get_nested_dict(rel, name):
  # This evaluates the string that describes the dictionary, as a dictionary
  # definition
  ret = rel.copy()
  # ret[name] = rel[name].map(lambda x: ast.literal_eval(x) if len(x) else np.NaN)
  ret = ret.dropna()
  # This joins rows on the index
  return ret.drop(columns=name).join(pd.DataFrame(ret[name].tolist()))

def get_nested_list(rel, name):
  ret = rel.copy()
  ret = ret.dropna().explode(name).dropna()
  ret = ret.join(pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()
  return ret.rename(columns={0: name})

def get_nested_list_dict(rel, name):
  ret = rel.copy()

  ret = ret.dropna().explode(name)

  exploded_pairs = pd.DataFrame(ret.apply(lambda x: {'_id': x['_id']} | x[name] if isinstance(x[name], dict) else {'_id': x['_id']}, axis=1).tolist())

  return ret.merge(exploded_pairs, on='_id').drop(columns=name)

names_df = get_nested_dict(people_df[['_id','name']], 'name')

experience_df = get_nested_list_dict(people_df[['_id','experience']], 'experience')

people_df = people_df.drop(columns=['name','education','group','skills','experience','honors','events','specilities','interests']).merge(names_df,on='_id')

In [ ]:
experience_df

,_id,org,title,end,start,desc
0,1,Johnson and Johnson,"Senior Scientist, Oncology Biomarkers",Present,November 2009,Biomarker Leader for compounds in clinical dev...
1,1,Albert Einstein Medical Center,Associate at Dept of Molecular Genetics,NaN,September 2008,Single Cell Gene expression.
2,1,Columbia University,Associate Research Scientist,NaN,August 2006,Work on peptide to restore wt p53 function in ...
3,1,Memorial Sloan Kettering Cancer Center,Post Doctoral Research Fellow,NaN,January 2003,Molecular profiling of colorectal cancer.
4,1,Sahlgrenska University Hospital,Research Scientist,NaN,November 2001,Cancer Research at Dept of Surgery.Molecular p...
...,...,...,...,...,...,...
164045,4998,Sona Koyo Steering Systems Ltd,Deptt. Head - HRD,NaN,October 2005,* Integration of HR Policies & Practices at va...
164046,4998,Caparo Group - India,Officer - Corporate & Strategic HR,NaN,March 2005,NaN
164047,4998,Roop Automotives Ltd,Executive - HR & Admin,NaN,May 2003,* Recruitment* Induction* Training & Developme...
164048,4998,North Bihar Nature & Adventure Foundation,Coordinator - HR & PR,NaN,September 1999,* Recruitment* Training & Development* General...


Now let's make some changes to convert people IDs to integers, and create IDs for the organizations they've worked for.

In [ ]:
organizations_df = experience_df['org'].drop_duplicates()

org_id_start = len(people_df)

organizations_df = organizations_df.reset_index().rename(columns={'org':'name'})
organizations_df['index'] = organizations_df['index'].apply(lambda x: x + org_id_start)
organizations_df

,index,name
0,5000,Johnson and Johnson
1,5001,Albert Einstein Medical Center
2,5002,Columbia University
3,5003,Memorial Sloan Kettering Cancer Center
4,5004,Sahlgrenska University Hospital
...,...,...
17201,169014,Honda Siel Cars India Ltd.
17202,169015,Sona Koyo Steering Systems Ltd
17203,169016,Caparo Group - India
17204,169017,Roop Automotives Ltd


Here is a relation of edges now.

In [ ]:
edges_df = experience_df[['_id','org']].merge(organizations_df,left_on='org',right_on='name').\
  drop(columns=['name','org']).rename(columns={'_id': 'from', 'index': 'to'})

edges_reversed_df = edges_df.rename(columns={'from': 'to', 'to': 'from'})

edges_df = pd.concat([edges_df, edges_reversed_df]).drop_duplicates()
edges_df

,from,to
0,1,5000
1,1,5001
2,1,5002
3,1,5003
4,1,5004
...,...,...
164015,169015,4998
164016,169016,4998
164017,169017,4998
164018,169018,4998


In [ ]:
max(edges_df['from'])

169018

In [ ]:
max(edges_df['to'])

169018

For PageRank we need to scale the weight transfer by the number of out-edges. To do that let's count the number of out-links for each node.

In [ ]:
outlinks_df = edges_df.groupby('from').size().to_frame('outlinks')
outlinks_df

,outlinks
from,
1,5
2,2
3,23
5,10
6,2
...,...
169014,1
169015,1
169016,1


Now we can see the whole process.

In [ ]:
weight_transfer_df = edges_df.merge(outlinks_df, left_on='from', right_index=True)
weight_transfer_df['weight'] = 1 / weight_transfer_df['outlinks']
weight_transfer_df

,from,to,outlinks,weight
0,1,5000,5,0.200000
1,1,5001,5,0.200000
2,1,5002,5,0.200000
3,1,5003,5,0.200000
4,1,5004,5,0.200000
...,...,...,...,...
164015,169015,4998,1,1.000000
164016,169016,4998,1,1.000000
164017,169017,4998,1,1.000000
164018,169018,4998,1,1.000000


In [ ]:
weight_transfer_df = weight_transfer_df.drop(columns=['outlinks']).dropna()
weight_transfer_df

,from,to,weight
0,1,5000,0.200000
1,1,5001,0.200000
2,1,5002,0.200000
3,1,5003,0.200000
4,1,5004,0.200000
...,...,...,...
164015,169015,4998,1.000000
164016,169016,4998,1.000000
164017,169017,4998,1.000000
164018,169018,4998,1.000000


## Convert to a Matrix

Now let's convert to a matrix...

In [ ]:
len(weight_transfer_df)


39798

In [ ]:
import numpy as np

max_id = max(organizations_df['index'])

weight_transfer_mat = np.zeros((max_id+1, max_id+1))

for i, row in weight_transfer_df.iterrows():
  weight_transfer_mat[int(row['to']), int(row['from'])] = row['weight']



In [ ]:
alpha = 0.85
beta = 1 - alpha

# Initialize PageRank to 1/n
pr = np.ones(weight_transfer_mat.shape[0]) / weight_transfer_mat.shape[0]

for i in range(5):
  pr = beta * (weight_transfer_mat @ pr) + alpha / weight_transfer_mat.shape[0]

In [ ]:
pr

array([5.02902041e-06, 8.35375629e-06, 6.29335375e-06, ...,
       5.27334852e-06, 5.27334852e-06, 5.27334852e-06])

## Exercise

Can you remove all *self-loops* from the graph, and recompute the PageRank?

In [ ]:
# TODO

In [ ]:
grader.grade('pagerank', pr[0:1000])

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
